In [1]:
import os
import numpy
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas

In [8]:
IMG_SIZE = 256
sigmaX = 30

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[numpy.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][numpy.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][numpy.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][numpy.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][numpy.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = numpy.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    return image

In [6]:
aptos_df = pandas.read_csv("../input/aptos2019-blindness-detection/train.csv")
dr_df = pandas.read_csv("../input/diabetic-retinopathy-resized/trainLabels.csv")
print(aptos_df['diagnosis'].value_counts())
print(dr_df['level'].value_counts())

0    1805
2     999
1     370
4     295
3     193
Name: diagnosis, dtype: int64
0    25810
2     5292
1     2443
3      873
4      708
Name: level, dtype: int64


In [9]:
aptos_df = pandas.read_csv("../input/aptos2019-blindness-detection/train.csv")
dr_df = pandas.read_csv("../input/diabetic-retinopathy-resized/trainLabels.csv")
aptos_dir = "../input/aptos2019-blindness-detection/train_images/"
dr_dir = "../input/diabetic-retinopathy-resized/resized_train/resized_train/"
images = []
labels = []

for idx in tqdm(range(len(aptos_df))):
    if aptos_df.iloc[idx]['diagnosis'] in [0, 2]:
        images.append(load_ben_color(aptos_dir+aptos_df.iloc[idx]['id_code']+'.png'))
        labels.append(aptos_df.iloc[idx]['diagnosis'])

100%|██████████| 3662/3662 [09:19<00:00,  6.55it/s]


In [10]:
count = 0
for idx in tqdm(range(len(dr_df))):
    if dr_df.iloc[idx]['level'] in [3, 4]:
        images.append(load_ben_color(dr_dir+dr_df.iloc[idx]['image']+'.jpeg'))
        labels.append(dr_df.iloc[idx]['level'])
    elif dr_df.iloc[idx]['level'] == 1 and count < 1022:
        images.append(load_ben_color(dr_dir+dr_df.iloc[idx]['image']+'.jpeg'))
        labels.append(dr_df.iloc[idx]['level'])
        count += 1

100%|██████████| 35126/35126 [02:11<00:00, 267.71it/s]


In [11]:
from tensorflow.keras.utils import to_categorical

# images = numpy.load('../input/aptosnumpy/images.npy')
images = numpy.array(images)
labels = numpy.array(labels)
cat_labels = to_categorical(labels)

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(images, cat_labels, 
                                                    test_size=0.1, shuffle=True,
                                                    random_state=42, stratify=labels)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
#     zca_whitening=True,
    zoom_range=[0.5,1.0],
    vertical_flip=True,
    horizontal_flip=True)
train_datagen.fit(x_train)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
#     zca_whitening=True,
    zoom_range=[0.5,1.0],
    vertical_flip=True,
    horizontal_flip=True)
test_datagen.fit(x_test)

In [ ]:
numpy.save('images.npy', images)

In [ ]:
from collections import Counter
counter = Counter(labels)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
d_class_weights = dict(enumerate(class_weights))
d_class_weights

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', numpy.unique(labels), labels)
d_class_weights = dict(enumerate(class_weights))
d_class_weights

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D

model = Xception(include_top=False, input_shape=(256, 256, 3))
x = GlobalAveragePooling2D()(model.output)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dense(254, activation='relu')(x)
x = Dense(64, activation='relu')(x)
op = Dense(5, activation='softmax')(x)

model = Model(inputs=model.inputs, outputs=op)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)


model.fit(train_datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=32,
          validation_data = (train_datagen.flow(x_train, y_train, batch_size=32)),
          callbacks=[checkpoint, earlystopping], verbose=1, 
          class_weight=d_class_weights)

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D

model = Xception(include_top=False, input_shape=(256, 256, 3))
x = GlobalAveragePooling2D()(model.output)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(32, activation='relu')(x)
op = Dense(5, activation='softmax')(x)

model = Model(inputs=model.inputs, outputs=op)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)


model.fit(train_datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=32,
          validation_data = (train_datagen.flow(x_train, y_train, batch_size=32)),
          callbacks=[checkpoint, earlystopping], verbose=1)#, class_weight=d_class_weights)

In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Activation
from tensorflow.keras import losses
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from sklearn.model_selection import GridSearchCV
import keras
import tensorflow

In [ ]:
def Randomized_Model(lr=0.01,dropout=0.5,optimizer="adam",loss='categorical_crossentropy',
                    last_activation="softmax",activation="relu",clipnorm=0.1,
                    decay=1e-2,momentum=0.5,l1=0.01,l2=0.001,No_of_CONV_and_Maxpool_layers=3,
                    No_of_Dense_Layers=3,No_of_Units_in_dense_layers=24,Conv2d_filters=60):
    
    model=Sequential()
    
    for i in range(0,No_of_CONV_and_Maxpool_layers):
        model.add(Conv2D(Conv2d_filters, (2,2) ,padding="same"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Flatten())
    
    for i in range (0,No_of_Dense_Layers):
        model.add(Dense(units=No_of_Units_in_dense_layers,activation=activation, kernel_regularizer=regularizers.l2(l2),
                  activity_regularizer=regularizers.l1(l1)))
    
    model.add(Dropout(dropout))
    model.add(Dense(units=64,activation=activation))
    
    model.add(Dense(units=5,activation=last_activation))
    if optimizer=="Adam":
        opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adagrad":
        opt=keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    if optimizer=="sgd":
        opt=keras.optimizers.SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
    
    model.compile(loss=loss,optimizer=opt,
                 metrics=['accuracy'])

    return model

In [ ]:
params = {'lr': (0.1, 0.01,1,0.001 ),
     'epochs': [10,5,15,30],
     'dropout': (0, 0.40, 0.8),
     'optimizer': ["Adam","Adagrad","sgd"],
     'loss': ["categorical_crossentropy"],#,"mean_squared_error","mean_absolute_error"],
     'last_activation': ["softmax"],#,"sigmoid"],
     'activation': ["relu","selu","tanh"],
     'clipnorm':(0.0,0.5,1),
     'decay':(1e-6,1e-4,1e-2),
     'momentum':(0.9,0.5,0.2),
     'l1': (0.01,0.001,0.0001),
     'l2': (0.01,0.001,0.0001),
     'No_of_CONV_and_Maxpool_layers':[2,3,4],
     'No_of_Dense_Layers': [2,3,4,5],
     'No_of_Units_in_dense_layers':[256,128,64,512],
     
     'Conv2d_filters':[60,40,80,120,220]
    }

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
# model class to use in the scikit random search CV 
model = KerasClassifier(build_fn=Randomized_Model, epochs=10, batch_size=20, verbose=1)
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=params, 
                          verbose=20,  n_iter=10, n_jobs=1)

In [ ]:
model = Xception(include_top=False, input_shape=(256, 256, 3))
x = GlobalAveragePooling2D()(model.output)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(32, activation='relu')(x)
op = Dense(5, activation='softmax')(x)

model = Model(inputs=model.inputs, outputs=op)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

checkpoint = tensorflow.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)


model.fit(train_datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=32,
          validation_data = (train_datagen.flow(x_train, y_train, batch_size=32)),
          callbacks=[checkpoint, earlystopping], verbose=1)

In [ ]:
from sklearn.metrics import accuracy_score

y=grid_result.predict(X_test)
yy = to_categorical(y)
random=accuracy_score(yy, y_test)
print("Base Accuracy ",random)

best_random = grid_result.best_estimator_
y1=best_random.predict(X_test)
y1y1 = to_categorical(y1)
Best=accuracy_score(y1y1, y_test)
print("Best Accuracy " ,Best)

print('Improvement of {:0.2f}%.'.format( 100 * (Best - random) / random))

In [ ]:
from sklearn.svm import SVC
clf = SVC()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images_, labels, test_size=0.1, random_state=42, shuffle=True, stratify=cat_labels)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)

In [ ]:
neigh.fit(X_train, y_train)

In [ ]:
y_pred2=neigh.predict(X_test)
accuracy_score(y_test,y_pred2)